In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 5000
load_target = 15

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [4]:
network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 464.72it/s]
INFO:linopy.io: Writing time: 0.52s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [8e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
48184 rows, 42352 cols, 127032 nonzeros  0s
38071 rows, 32239 cols, 117366 nonzeros  0s
37991 rows, 32159 cols, 117722 nonzeros  0s

Solving MIP model with:
   37991 rows
   32159 cols (0 binary, 5 integer, 0 implied int., 32154 continuous)
   117722 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.2s
 S       0       0         0   0.00%   -inf            22965811786.85     Large        0      0      0         0     8.7s
 R       0       0         

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 105141 duals
Objective: 2.29e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


es             2
  LP iterations     59309 (total)
                    604 (strong br.)
                    758 (separation)
                    13062 (heuristics)
Writing the solution to /tmp/linopy-solve-vfwnia2g.sol


('ok', 'optimal')

In [6]:
network.statistics().round(0)

Optimal Capacity  Installed Capacity      Supply  \
Generator backstop            2210.0                 0.0    492081.0   
          biogas               500.0                 0.0    655134.0   
          offwind                0.0                 0.0         0.0   
          onwind              5986.0                 0.0   7836667.0   
          solar               6418.0                 0.0   6645317.0   
Link      AC                  4495.0                 0.0  14186701.0   
          biogas               500.0                 0.0    655134.0   
          h2                   700.0                 0.0    206763.0   
          li-ion              1908.0                 0.0   2518616.0   
          mixedgas             600.0                 0.0    435388.0   
Load      -                      0.0                 0.0         0.0   
Store     h2                  3146.0                 0.0    103382.0   
          li-ion             18704.0                 0.0   2598095.0   

                    Withdrawal    Dispatch  Transmission  Capacity Factor  \
Generator backstop         0.0    492081.0           0.0              0.0   
          biogas           0.0    655134.0           0.0              0.0   
          offwind          0.0         0.0           0.0              NaN   
          onwind           0.0   7836667.0           0.0              0.0   
          solar            0.0   6645317.0           0.0              0.0   
Link      AC        14298785.0   -112084.0    11553915.0              0.0   
          biogas      655134.0         0.0      655134.0              0.0   
          h2          286580.0    -79817.0           0.0              0.0   
          li-ion     2625839.0   -107224.0           0.0              0.0   
          mixedgas    758516.0   -323128.0      758516.0              0.0   
Load      -         15000000.0 -15000000.0           0.0              NaN   
Store     h2          103382.0         0.0           0.0              0.0   
          li-ion     2605042.0     -6947.0           0.0              1.0   

                    Curtailment  Capital Expenditure  Operational Expenditure  \
Generator backstop          0.0         0.000000e+00             5.172758e+09   
          biogas            0.0         0.000000e+00             8.370916e+08   
          offwind           0.0         0.000000e+00             0.000000e+00   
          onwind     12232143.0         8.187064e+09             2.255360e+08   
          solar        605368.0         3.333884e+09             0.000000e+00   
Link      AC                0.0         3.904260e+08             0.000000e+00   
          biogas            0.0         0.000000e+00             0.000000e+00   
          h2                0.0         1.091117e+08             0.000000e+00   
          li-ion            0.0         0.000000e+00             0.000000e+00   
          mixedgas          0.0         7.989853e+08             4.881469e+07   
Load      -                 0.0         0.000000e+00             0.000000e+00   
Store     h2                0.0         2.398146e+08             0.000000e+00   
          li-ion            0.0         3.602315e+09            -1.948910e+05   

                    Revenue  Market Value  
Generator backstop      0.0           NaN  
          biogas        0.0           NaN  
          offwind       0.0           NaN  
          onwind        0.0           NaN  
          solar         0.0           NaN  
Link      AC            0.0           NaN  
          biogas        0.0           NaN  
          h2            0.0           NaN  
          li-ion        0.0           NaN  
          mixedgas      0.0           NaN  
Load      -             0.0           0.0  
Store     h2            0.0           NaN  
          li-ion        0.0           NaN